# RFM Customer Segmentation — Online Retail (Power BI Ready)

Notebook ini disiapkan untuk **Portofolio → 2_Customer_Segmentation** dengan struktur folder:
- `Portofolio/2_Customer_Segmentation/data_raw/Online Retail.xlsx`
- `Portofolio/2_Customer_Segmentation/data_clean/`
- `Portofolio/2_Customer_Segmentation/outputs/`

> Catatan: Jalankan sel secara berurutan dari atas ke bawah.

## 0) Mount Google Drive & Setup Path

In [ ]:
# --- Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- Base paths (sesuaikan jika perlu) ---
BASE = '/content/drive/MyDrive/Portofolio/2_Customer_Segmentation'
DATA_RAW = f'{BASE}/data_raw/Online Retail.xlsx'
DATA_CLEAN_DIR = f'{BASE}/data_clean'
OUTPUTS_DIR = f'{BASE}/outputs'

# --- Create dirs if not exist ---
import os
os.makedirs(DATA_CLEAN_DIR, exist_ok=True)
os.makedirs(OUTPUTS_DIR, exist_ok=True)

print('RAW exists:', os.path.exists(DATA_RAW))
print('Clean dir :', DATA_CLEAN_DIR)
print('Outputs   :', OUTPUTS_DIR)

## 1) Install/Import Packages

In [ ]:
!pip -q install openpyxl pandas numpy
import pandas as pd, numpy as np
from datetime import datetime, timedelta

## 2) Load & Quick Inspect

In [ ]:
# Baca excel
df = pd.read_excel(DATA_RAW, engine='openpyxl')
print('Shape raw:', df.shape)
df.head()

## 3) Basic Cleaning

In [ ]:
# Pastikan kolom yang diperlukan ada
required_cols = ['InvoiceNo','StockCode','Description','Quantity','InvoiceDate','UnitPrice','CustomerID','Country']
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f'Kolom hilang di source: {missing}')

# 3.1 Tipe data & normalisasi dasar
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
df['CustomerID'] = pd.to_numeric(df['CustomerID'], errors='coerce').astype('Int64')

# 3.2 Drop baris tanpa tanggal / customer
df = df.dropna(subset=['InvoiceDate', 'CustomerID'])

# 3.3 Buang transaksi return (InvoiceNo diawali "C") — opsional
mask_credit = df['InvoiceNo'].astype(str).str.startswith('C')
df = df[~mask_credit]

# 3.4 Buang nilai Qty <= 0 atau UnitPrice <= 0
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# 3.5 TotalPrice
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# 3.6 Hapus duplikat baris persis sama
df = df.drop_duplicates()

print('Shape after clean:', df.shape)
df.head()

## 4) Save Cleaned to Parquet (untuk Power BI)

In [ ]:
clean_path = f'{DATA_CLEAN_DIR}/online_retail_clean.parquet'
df.to_parquet(clean_path, index=False)
clean_path

## 5) RFM Feature Engineering

In [ ]:
# Snapshot date (keesokan hari dari transaksi terakhir)
snapshot_date = df['InvoiceDate'].max().normalize() + pd.Timedelta(days=1)

rfm = (df.groupby('CustomerID')
         .agg(LastPurchase=('InvoiceDate','max'),
              Frequency=('InvoiceNo', pd.Series.nunique),
              Monetary=('TotalPrice','sum'))
         .reset_index())

rfm['Recency'] = (snapshot_date - rfm['LastPurchase']).dt.days

# Urutan kolom
rfm = rfm[['CustomerID','Recency','Frequency','Monetary','LastPurchase']]
rfm.head()

## 6) RFM Scoring (quintiles 1–5)

In [ ]:
def r_score(x):
    # Recency: lebih kecil lebih baik -> score tinggi untuk nilai kecil
    return pd.qcut(x, 5, labels=[5,4,3,2,1])

def fm_score(x):
    # Frequency/Monetary: lebih besar lebih baik
    return pd.qcut(x.rank(method='first'), 5, labels=[1,2,3,4,5])

rfm['R'] = r_score(rfm['Recency']).astype(int)
rfm['F'] = fm_score(rfm['Frequency']).astype(int)
rfm['M'] = fm_score(rfm['Monetary']).astype(int)

rfm['RFM_Score'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)
rfm.head()

## 7) Simple Segment Mapping

In [ ]:
def segment_map(row):
    r,f = row['R'], row['F']
    if r >= 4 and f >= 4:
        return 'Champions'
    if r >= 4 and f <= 2:
        return 'New Customers'
    if r == 3 and f >= 4:
        return 'Loyal'
    if r <= 2 and f >= 4:
        return 'At Risk but Loyal'
    if r <= 2 and f <= 2:
        return 'Hibernating'
    return 'Potential Loyalist'

rfm['Segment'] = rfm.apply(segment_map, axis=1)
rfm['AOV'] = (rfm['Monetary'] / rfm['Frequency']).replace([np.inf, -np.inf], np.nan)

rfm.head(10)

## 8) Export CSV (untuk Power BI & Dokumentasi)

In [ ]:
# Join ringkas dengan negara (opsional)
cust_country = (df[['CustomerID','Country']]
                .drop_duplicates(subset=['CustomerID'])
                .set_index('CustomerID'))

rfm_export = rfm.join(cust_country, on='CustomerID')
rfm_export_path = f'{OUTPUTS_DIR}/rfm_summary.csv'
rfm_export.to_csv(rfm_export_path, index=False)
rfm_export_path, rfm_export.shape

## 9) Insight Ringkas

In [ ]:
insight = {}
insight['total_customers'] = int(rfm['CustomerID'].nunique())
insight['revenue_total'] = float(df['TotalPrice'].sum())
insight['avg_aov'] = float((rfm['Monetary'] / rfm['Frequency']).mean())
insight['segment_counts'] = rfm['Segment'].value_counts().to_dict()

insight